In [51]:
import pandas as pd
import sqlite3


In [52]:
orders = pd.read_csv("orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [53]:
users = pd.read_json("users.json")
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [54]:
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

with open("restaurants.sql", "r") as f :
  sql_script = f.read()

cursor.executescript(sql_script)

restaurants = pd.read_sql_query("SELECT *  from restaurants", conn)
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [55]:
merged1 = pd.merge(orders, users, on="user_id", how="left")
final_df = pd.merge(merged1, restaurants, on="restaurant_id", how="left")

final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [56]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


In [57]:
gold_df = final_df[final_df["membership"] == "Gold"]
gold_df.groupby("city")["total_amount"].sum().sort_values(ascending=False)


,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [58]:
final_df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)


,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [59]:
#q1
final_df[final_df["membership"]=="Gold"] \
    .groupby("city")["total_amount"] \
    .sum().sort_values(ascending=False)


,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [60]:
#q2
final_df.groupby("cuisine")["total_amount"] \
    .mean().sort_values(ascending=False)


,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [61]:
#q3
user_total = final_df.groupby("user_id")["total_amount"].sum()
count_users = (user_total > 1000).sum()
print(count_users)


2544


In [62]:
#q4
bins = [3.0,3.5,4.0,4.5,5.0]
labels = ["3.0–3.5","3.6–4.0","4.1–4.5","4.6–5.0"]

final_df["rating_range"] = pd.cut(final_df["rating"],
                                  bins=bins, labels=labels, include_lowest=True)

final_df.groupby("rating_range")["total_amount"] \
    .sum().sort_values(ascending=False)


/tmp/ipython-input-808920955.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final_df.groupby("rating_range")["total_amount"] \


,total_amount
rating_range,
4.6–5.0,2197030.75
3.0–3.5,2136772.70
4.1–4.5,1960326.26
3.6–4.0,1717494.41


In [63]:
#q5
final_df[final_df["membership"]=="Gold"] \
    .groupby("city")["total_amount"] \
    .mean().sort_values(ascending=False)


,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [64]:
#q6
pd.DataFrame({
    "restaurants": final_df.groupby("cuisine")["restaurant_id"].nunique(),
    "revenue": final_df.groupby("cuisine")["total_amount"].sum()
}).sort_values("restaurants")


,restaurants,revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [65]:
#q7
round((final_df["membership"].eq("Gold").mean())*100)


50

In [66]:
#q8
g = final_df.groupby("restaurant_name_y") \
    .agg(avg_value=("total_amount","mean"),
         orders=("order_id","count"))

result = g[g["orders"] < 20] \
    .sort_values("avg_value", ascending=False)

print(result.head())


                     avg_value  orders
restaurant_name_y                     
Restaurant_294     1040.222308      13
Restaurant_262     1029.473333      18
Restaurant_77      1029.180833      12
Restaurant_193     1026.306667      15
Restaurant_7       1002.140625      16


In [67]:
print(final_df.columns)


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating', 'rating_range'],
      dtype='object')


In [68]:

top_rest_id = "Restaurant_294"

final_df[final_df["restaurant_name_y"] == top_rest_id] \
    [["restaurant_name_y", "restaurant_name_x", "name", "cuisine", "city"]] \
    .drop_duplicates()


,restaurant_name_y,restaurant_name_x,name,cuisine,city
1407,Restaurant_294,Hotel Dhaba Multicuisine,User_2895,Italian,Chennai
1643,Restaurant_294,Hotel Dhaba Multicuisine,User_1363,Italian,Bangalore
2426,Restaurant_294,Hotel Dhaba Multicuisine,User_2944,Italian,Hyderabad
3174,Restaurant_294,Hotel Dhaba Multicuisine,User_315,Italian,Chennai
3243,Restaurant_294,Hotel Dhaba Multicuisine,User_2100,Italian,Hyderabad
4007,Restaurant_294,Hotel Dhaba Multicuisine,User_2137,Italian,Pune
4999,Restaurant_294,Hotel Dhaba Multicuisine,User_1478,Italian,Pune
5157,Restaurant_294,Hotel Dhaba Multicuisine,User_701,Italian,Hyderabad
5718,Restaurant_294,Hotel Dhaba Multicuisine,User_1223,Italian,Chennai
5851,Restaurant_294,Hotel Dhaba Multicuisine,User_2089,Italian,Hyderabad


In [69]:
combo_revenue = final_df.groupby(["membership", "cuisine"])["total_amount"] \
    .sum().reset_index()

combo_revenue = combo_revenue.sort_values("total_amount", ascending=False)
combo_revenue


,membership,cuisine,total_amount
7,Regular,Mexican,1072943.30
6,Regular,Italian,1018424.75
3,Gold,Mexican,1012559.79
2,Gold,Italian,1005779.05
5,Regular,Indian,992100.27
1,Gold,Indian,979312.31
0,Gold,Chinese,977713.74
4,Regular,Chinese,952790.91


In [77]:

final_df["order_date"] = pd.to_datetime(final_df["order_date"])

final_df["quarter_num"] = final_df["order_date"].dt.quarter

q_rev = final_df.groupby("quarter_num")["total_amount"] \
    .sum().sort_values(ascending=False)


q_rev


,total_amount
quarter_num,
3,2037385.10
4,2018263.66
1,2010626.64
2,1945348.72


In [71]:
gold_orders = final_df[final_df["membership"] == "Gold"].shape[0]
print(gold_orders)


4987


In [72]:
hyd_revenue = final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum()
print(round(hyd_revenue))


1889367


In [73]:
distinct_users = final_df["user_id"].nunique()
print(distinct_users)


2883


In [74]:
avg_gold = final_df[final_df["membership"] == "Gold"]["total_amount"].mean()
print(round(avg_gold, 2))


797.15


In [75]:
high_rating_orders = final_df[final_df["rating"] >= 4.5].shape[0]
print(high_rating_orders)


3374


In [76]:
gold_df = final_df[final_df["membership"] == "Gold"]

top_city = gold_df.groupby("city")["total_amount"].sum().idxmax()

orders_in_top_city = gold_df[gold_df["city"] == top_city].shape[0]

print(top_city, orders_in_top_city)


Chennai 1337
